In [ ]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2023.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# Getting Started

This notebook outlines the best operating practices for mapping a given quantum circuit to real IBM backends. The tools used within build on Qiskit yet not all are contained within this repository. This repository also contains custom transpiler passes that users may construct their own `PassManager`s from. 

In [4]:
from qiskit import IBMQ, transpile
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.opflow import I, X, Z, PauliTrotterEvolution, Suzuki
from qiskit.providers.fake_provider import FakeMumbai

import numpy as np
import matplotlib.pyplot as plt
# plt.style.use("default")
plt.style.use("dark_background")

In [5]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub="", group="", project="")

ibmqfactory.load_account:WARNING:2022-11-30 13:25:05,780: Credentials are already in use. The existing account in the session will be replaced.


## Problem: Quantum Simulation

As an example, we'll use the problem statement from [this manuscript](https://arxiv.org/abs/2108.09197), the quantum simulation of an Ising model Hamiltonian:
$$
H = -J \sum_{\langle i,j \rangle} Z_i Z_j + h \sum_i X_i
$$
where $J$ is the exchange coupling between adjacent spins and $h$ is the transverse magnetic field. Here $X_i$ and $Z_j$ are the Pauli matrices acting on qubits $i$ and $j$, respectively. This model describes an interacting system of spins in a magnetic field, which is normally a nearest-neighbor interaction, in order to highlight the mapping of this problem to quantum hardware, we make it an all-to-all interaction. <br> <br>

We will use tools from `qiskit.opflow` to generate the circuits needed for the simulation. The Hamiltonian $H$ is formed by instantiating `Parameter`s and building the interactions from the Pauli matrices `I`, `X`, and `Z` by concatenating them in tensor products ($\otimes$) represented in `opflow` by the caret symbol `^`. The formal solution to Schrödinger's equation is
$$
U = e^{-iHt},
$$
the *time-evolution unitary* corresponding to Hamiltonian evolution under $H$, and is found formally using the `.exp_i()` on an operator expression.

In [6]:
num_spins = 3

JJ = Parameter("J")
hh = Parameter("h")
tt = Parameter("t")

ham = -JJ * sum(
    [
        sum(
            [
                (I ^ idx) ^ Z ^ (I ^ jdx) ^ Z ^ (I ^ (num_spins - idx - jdx - 2))
                for jdx in range(num_spins - idx - 1)
            ]
        )
        for idx in range(num_spins - 1)
    ]
) + hh * sum([(I ^ idx) ^ X ^ (I ^ (num_spins - idx - 1)) for idx in range(num_spins)])
U_ham = (ham * tt).exp_i()
print(U_ham)

## Converting Operators to Circuits

The `qiskit.opflow` module contains methods to convert the time-evolved operators (`EvolvedOp`s) to circuits. One common method is the Suzuki-Trotter decomposition, in which the total evolution time $t$ is broken into `num_steps` $N$. By choosing the second-order of the `PauliTrotterEvolution`, we create a circuit that acts like our unitary to second order $\mathcal{O}((t/N)^2)$. 

Practically, since many of the following transpilation steps are computationally intensive, it may make sense to break up your circuit into smaller subcircuits, and then combine them together to get the final circuit. This is naturally acheived for Trotterized algorithms, since `reps` here just repeats the same circuit `num_steps` times, hence we just set `num_steps=1` and the resulting circuits can be combined into as many steps as desired later.

In [11]:
num_steps = 1
trot_circ = (
    PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=num_steps))
    .convert(U_ham)
    .to_circuit()
)
# trot_circ.draw("mpl")
# trot_circ.decompose().draw("mpl")
# trot_circ.decompose().decompose().draw("mpl")

## Transpile for Good SWAP Mapping

In general problems must respect the layout of the actual quantum hardware. Due to limited connectivity, this often entails doing SWAP operations to move quantum information around. SWAPs are costly in the sense they consist of three `CX`s. The Qiskit transpiler with `optimization_level=3` uses the [SABRE SWAP method](https://arxiv.org/abs/1809.02573), which is efficient, however stochastic, since the SWAP-mapping problem is NP-hard. Here we do it several time and take the solution with the lowest CNOT count. This operation serves only to minimize the number of SWAPs in the transpiled circuits, and is not aware of noise on the underlying qubits. That is considered in a following step.

In [12]:
# TODO - check issue with floats not being rounded off

num_tries = 10

# backend = FakeMumbai()
# backend = provider.get_backend('ibmq_mumbai')
backend = provider.get_backend("ibm_lagos")
trot_circ_ts = transpile(
    [trot_circ] * num_tries, backend, optimization_level=3, seed_transpiler=12345
)
cx_counts = [trot_circ_ts[idx].count_ops()["cx"] for idx in range(num_tries)]
print(cx_counts)

/opt/anaconda3/envs/qiskit-dev/lib/python3.9/site-packages/numpy/linalg/linalg.py:2139: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/anaconda3/envs/qiskit-dev/lib/python3.9/site-packages/numpy/linalg/linalg.py:2139: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


[18, 18, 18, 18, 18, 18, 18, 18, 18, 18]


In [13]:
best_idx = np.argmin(cx_counts)
trot_circ_t = trot_circ_ts[best_idx]

## Noise-aware Layout

Now that we have a SWAP-mapped optimal circuit, we consider the layouts on the actual quantum backend. These layouts are found by the VF2 subgraph isomorphism algorithm, which is very fast. Since the transpiler mapped the circuit to physical qubits, we must first "deflate" the circuits with `deflate_circuit` (which removes idle qubits), finds the layouts with `matching_layouts`, then scores those layouts due to error rates, which are calculated by a cost function that may be specified by the user, see [`mapomatic`](https://github.com/Qiskit-Partners/mapomatic) documentation for how. The default cost function includes errors determined for each qubit gates and measurements, although not decoherence/relaxation caused by idle time, producing an *infidelity score* where the lowest number is the preferred layout.

In [14]:
from mapomatic import deflate_circuit, evaluate_layouts, matching_layouts

trot_circ_def = deflate_circuit(trot_circ_t)
layouts = matching_layouts(trot_circ_def, backend)
scored_layouts = evaluate_layouts(
    trot_circ_def, layouts, backend
)  # cost_function = cost_func
print(scored_layouts)

ImportError: cannot import name 'get_ecr_pairs_from_backend' from 'qiskit_research.utils.pulse_scaling' (/Users/ntbronn/Documents/codebase/qiskit-research/qiskit_research/utils/pulse_scaling.py)

## Pulse Scaling

For certain problems, in particular Trotterized quantum simulation problems, or other algorithms that require small angles of rotation in the two-qubit Hilbert space, it is more efficient to implement operations in terms of pulses extracted from the CNOT gate. Basically, a CNOT gate is *locally-equivalent* to an $R_{ZX}(\pi/2)$ rotation, meaning it is built from that and single-qubit rotations, as can be seen from the following code:

In [ ]:
# TODO: check out Weyl decomp stuff

qc = QuantumCircuit(2)
qc.cx(0, 1)
qc_rzx = transpile(qc, basis_gates=["sx", "rz", "rzx"])

fig, (ax1, ax2) = plt.subplots(1, 2)
qc.draw("mpl", ax=ax1)
qc_rzx.draw("mpl", ax=ax2)

The `RZXGate` is very similar to the native two-qubit interation called [echoed cross resonance](https://arxiv.org/abs/1603.04821) which is used to create entanglement on IBM backends. In particular, many two-qubit interactions for quantum simulation, such as the $ZZ$-interaction of our Ising Hamiltonian, can be more efficiently represented (in terms of error) by $R_{ZX}(\theta)$ rotations, which are automatically broken into scaled echoed cross resonance `secr` gates unless `unroll_rzx_to_ecr` is set to `False`.

In [ ]:
from qiskit_research.utils.convenience import scale_cr_pulses

theta = Parameter("$\\theta$")

qc = QuantumCircuit(2)
qc.rzz(theta, 0, 1)
qc_cx = transpile(qc, basis_gates=["rz", "sx", "cx"])
qc_rzx = scale_cr_pulses(qc_cx, backend)  # unroll_rzx_to_ecr = True, param_bind = {}

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 6))
qc.draw("mpl", ax=ax1)
qc_cx.draw("mpl", ax=ax2)
qc_rzx.draw("mpl", ax=ax3)
ax2.set_title("These are all equivalent circuits")

When we are implementing two-qubit rotation angles $\theta$ less than $\pi/2$, we can more efficiently express these interaction in terms of $R_{ZX}(\theta)$ rotations and directly build them from scaled echoed cross resonance (`secr`($\theta$)) pulses obtained from the backend, as detailed in [this manuscript](http://arxiv.org/abs/2012.11660). This method first uses a greedy algorithm called [template opimization](http://arxiv.org/abs/1909.05270) to identify parts of the circuit that can be substituted by $R_{ZX}$ rotations. If parameters are passed to the method via `param_bind`, it will bind them to the circuit and attach the necessary pulse gates for implementing the $R_{ZX}$ rotations (otherwise they can be bound and scaled later with `attach_cr_pulses`). Below we will do them separately, because we will attach a series of `Parameter`s as a function of time, and it is more efficient to do the template optimization step once since it is greedy and attaching the pulse schedules in quick.

In [ ]:
# my_layout = scored_layouts[0][0]  # the layout with the lowest score (i.e., error)
my_layout = [8, 11, 14, 16] # debugging timing problem on 
trot_circ_sca = scale_cr_pulses(
    transpile(trot_circ_def, initial_layout=my_layout), backend
)
trot_circ_sca.draw("mpl", idle_wires=False)

In [ ]:
from qiskit_research.utils.convenience import attach_cr_pulses

num_time_steps = 51
t_range = np.linspace(0, 10, num_time_steps)  # values from manuscript
param_bind = {JJ: 0.5236, hh: 1}  # values from manuscript

circs = build_ising_circuits(trot_circ_sca, backend, num_trotter_steps, my_layout, param_bind, t_range)
# circs = build_ising_circuits(transpile(trot_circ_sca), backend, num_trotter_steps, [0, 1, 2, 3], param_bind, t_range)

In [ ]:
circs[-1].draw('mpl', idle_wires=False)

## Pauli Twirling

Pauli twirling is a form of randomized compiling that inserts pairs of Pauli gates (`I`, `X`, `Y`, `Z`) before and after entangling gates such that the overall unitary is the same, but the way it is implemented is different. This has the effect of turning coherent errors into stochastic errors, which can then be elimated by sufficient averaging. This is done a number of times (`num_twirled_circuits`) for the benefit of averaging. **Note:** we are probably using an insufficient basis set to currently cancel all errors.

In [ ]:
from qiskit_research.utils.convenience import add_pauli_twirls

num_twirls = 5
# this returns a circuit with shape len(circs) x num_twirled_circuits
twirled_circs = add_pauli_twirls(
    circs, num_twirled_circuits=num_twirls, seed=12345
)  # transpile_added_paulis = False

In [ ]:
twirled_circs[-1][-1].draw("mpl", idle_wires=False)

Look good! Now before proceeding to dynamical decoupling, we must convert to the native basis gates of the backend so that we can retrieve gate timing information, which is necessary to add dynamical decoupling passes. (Unless you set the keyword argument `transpile_added_paulis=True` in the above). You will also need to run this before running on a backend.

In [ ]:
from qiskit_research.utils.convenience import transpile_paulis

twirled_circs_t = transpile_paulis(twirled_circs)

## Dynamical Decoupling

Dynamical decoupling (DD) is a way of modifying the noise power spectrum $S(\omega)$ observed by qubits (see [this recent review](https://arxiv.org/abs/2207.03670)), and is typically implemented by a sequence of gates scheduled during a given qubit idle time that compose to the identity with specific delay times to fill the idle time in a calculated manner. Considerations for which sequences to use may involve decoherent error due to idle time versus single-qubit gate errors and/or crosstalk during two-qubit gates. Because the addition of gates is not always in the set of `basis_gates` defined by the backend, `add_pulse_cals=True` uses [Pulse Gates](https://qiskit.org/documentation/tutorials/circuits_advanced/05_pulse_gates.html) to add the correct implementation to the circuit with added DD.

In [ ]:
from qiskit_research.utils.convenience import add_dynamical_decoupling

# twirled_circs_with_dd = add_dynamical_decoupling(
circs_with_dd = add_dynamical_decoupling(
    # twirled_circs_t, backend, "XpXm", add_pulse_cals=True
    circs, backend, "X2pm", add_pulse_cals=True
)

In [ ]:
from qiskit.visualization import timeline_drawer

# this just displays a small range
# timeline_drawer(twirled_circs_with_dd[-1][-1], time_range=[1, 20000], show_idle=False)
timeline_drawer(circs_with_dd[0], time_range=[0, 1000], show_idle=False)

In [ ]:
from qiskit import schedule
from qiskit.pulse import ControlChannel, DriveChannel
circ_sched = schedule(circs_with_dd[0], backend)

In [ ]:
circ_sched.draw(time_range=[0, 1000])

In [ ]:
from qiskit.pulse import Play
for (time, inst) in circ_sched.instructions:
    if isinstance(inst, Play):
        print(time, inst)

In [ ]:
conf = backend.configuration()
defs = backend.defaults()
props = backend.properties()

In [ ]:
conf.control_channels

In [ ]:
d_chan = 8 # 14
u_chan = 18 # 29

for cidx, circ in enumerate(circs_with_dd):
     circ_sched = schedule(circ, backend)
     prev_end = 0
     # for (time, inst) in circ_sched.filter(channels=[DriveChannel(14), ControlChannel(29)]).instructions:
     for (time, inst) in circ_sched.filter(channels=[DriveChannel(d_chan), ControlChannel(u_chan)]).instructions:
          # print('Circuit: {}, Time: {}, Drive: {}, Ctrl: {}'.format(cidx, time, inst.ch_duration(DriveChannel(14)), inst.ch_duration(ControlChannel(29))))
          cur_time = time + inst.ch_duration(DriveChannel(d_chan)) + inst.ch_duration(ControlChannel(u_chan))
          if (inst.ch_duration(DriveChannel(d_chan)) + inst.ch_duration(ControlChannel(u_chan))) > 0:
               # print('{} - {}'.format(prev_end, time))
               if (time < prev_end):
                    print('Problem at time {} after time {} with circuit {} for Inst {}'.format(cur_time, prev_end, cidx, inst))
          
               prev_end = cur_time
          # if (time+inst.ch_duration(DriveChannel(14)) < prev_end):
          #      print('Problem at time {} with circuit {}'.format(prev_end, cidx))
          # prev_end = time + inst.ch_duration(DriveChannel(14))

In [ ]:
# from qiskit_research.utils.cost_funcs import cost_func_scaled_cr
# for circ in twirled_circs_with_dd[-1]:
#     print(evaluate_layouts(circ, scored_layouts[0][0], backend, cost_function=cost_func_scaled_cr))

## Circuit Execution

This runs the given circuits on the backend. This will be expanded to include different methods of running, i.e. Qiskit Runtime. 

In [ ]:
from qiskit.providers.ibmq.managed import IBMQJobManager

# # the backend only accepts a QuantumCircuit or List[QuantumCircuit]
# flattened_circs = [
#     # circ for circs in twirled_circs_with_dd for circ in circs
#     circ for circs in twirled_circs for circ in circs

# ] 

# Use Job Manager to break the circuits into multiple jobs.
job_manager = IBMQJobManager()
# ising_job_set = job_manager.run(circs, backend=backend, 
ising_job_set = job_manager.run(circs_with_dd, backend=backend, 
    name='Ising-t{}-Nt{}'.format(stop_time, num_trotter_steps), shots=5000)
# ising_job_set = job_manager.run(flattened_circs, backend=backend, name='Ising-t{}-Nt{}-sim'.format(stop_time, num_trotter_steps))

ising_job_set_id = ising_job_set.job_set_id()
print("Job Set ID: {}".format(ising_job_set_id))

In [ ]:
ising_job_set_id = '7aec82091817425880af55e9d8e5f6f6-16667976273163059'
# ising_job_sim_id = 'dc7f791b7b6f4e6bb09e094294f41019-16660448403650491'
# job_manager = IBMQJobManager()
# ising_job_set = job_manager.retrieve_job_set(job_set_id=ising_job_sim_id, provider=provider)

In [ ]:
# the backend only accepts a QuantumCircuit or List[QuantumCircuit]
flattened_circs = [
    circ for circs in twirled_circs_with_dd for circ in circs
]  # first 5 circs are same Pauli twirled circuit at same time
counts = backend.run(flattened_circs).result().get_counts()

## Measurement Error Mitigation

This uses `mthree` (matrix-free measurement mitigation) to do $LU$-decomposition on a readout calibration routine to efficently correct for readout errors. Note that `cals_from_system` runs an experiment on your chosen `backend` and then applies it to your results to calculate quasi-probabilities, with the default `num_shots=8192`.

In [ ]:
from mthree import M3Mitigation

mit = M3Mitigation(backend)
mit.cals_from_system(my_layout)

In [ ]:
# apply the correction
quasi_probs = mit.apply_correction(counts, my_layout)

In [ ]:
from helper import combine_twirled_data, combine_mag_data
twirled_test = combine_twirled_data(counts, num_twirls)
mag_test = combine_mag_data(twirled_test)

In [ ]:
counts

In [ ]:
from helper import combine_twirled_data, combine_mag_data

twirled_data = combine_twirled_data(quasi_probs, num_twirls)
mag_data = combine_mag_data(twirled_data)

In [ ]:
mag_vec_data = {"x": [], "y": [], "z": []}

for tidx, _ in enumerate(t_range):
    mag_vec_data["x"].append(sum([1-2*sum([value for key, value in mag_data["x"][tidx].items() 
        if key[num_spins-qidx-1] == '1']) for qidx, _ in enumerate(my_layout)])/num_spins)
    mag_vec_data["y"].append(sum([1-2*sum([value for key, value in mag_data["y"][tidx].items() 
        if key[num_spins-qidx-1] == '1']) for qidx, _ in enumerate(my_layout)])/num_spins)
    mag_vec_data["z"].append(sum([1-2*sum([value for key, value in mag_data['z'][tidx].items() 
        if key[num_spins-qidx-1] == '1']) for qidx, _ in enumerate(my_layout)])/num_spins)

In [ ]:
1-2*sum([value for key, value in mag_data['x'][0].items() if key[0] == '1'])

In [ ]:
1-2*sum([value for key, value in mag_data['x'][0].items() if key[0] == '1'])

In [ ]:
mag_data['y'][0]

In [ ]:
sum([value for key, value in mag_data['z'][0].items() if key[0] == '0'])

In [ ]:
pnts = [mag_vec_data["x"], mag_vec_data["y"], mag_vec_data["z"]]

plt.rcParams["figure.figsize"] = [7, 7]
plt.rcParams["figure.autolayout"] = True
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
x = np.cos(u) * np.sin(v)
y = np.sin(u) * np.sin(v)
z = np.cos(v)
ax.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha=0.25, antialiased=False)
ax.plot(pnts[0], pnts[1], pnts[2], color='r', lw=4.0, label='Exact')
ax.set_title('Net Magnetization', fontsize=20)
plt.axis('off')
plt.show()

In [ ]:
np.sqrt(np.asarray(mag_vec_data["x"])**2 + np.asarray(mag_vec_data["y"])**2 + np.asarray(mag_vec_data["z"])**2)

In [ ]:
# collect quasi-probabilities from different Pauli twirls
quasi_probs_twirled = []
for time_idx in range(num_time_steps):
    quasi_prob_twirled = {}
    for twidx in range(num_twirls):
        for key in quasi_probs[time_idx * num_twirls + twidx].keys():
            try:
                quasi_prob_twirled[key] += (
                    quasi_probs[time_idx * num_twirls + twidx][key] / num_twirls
                )
            except:
                quasi_prob_twirled[key] = (
                    quasi_probs[time_idx * num_twirls + twidx][key] / num_twirls
                )

    quasi_probs_twirled.append(quasi_prob_twirled)

In [ ]:
# separate list of dicts into dict of lists
quasi_probs_dict = {}
for time_step in quasi_probs_twirled:
    for key in time_step.keys():
        try:
            quasi_probs_dict[key].append(time_step[key])
        except:
            quasi_probs_dict[key] = [time_step[key]]

In [ ]:
# plot results
fig, ax = plt.subplots(figsize=(12, 4))
for key in quasi_probs_dict.keys():
    ax.plot(t_range, quasi_probs_dict[key], lw=2, label=key)
ax.set_xlabel("time step (arb)")
ax.set_ylabel("quasi-probability")
ax.legend(loc=4)
plt.show()

In [ ]:
from helper import exact_magnetization
mags = exact_magnetization(U_ham, param_bind, t_range)

In [ ]:
pnts = mags

plt.rcParams["figure.figsize"] = [7, 7]
plt.rcParams["figure.autolayout"] = True
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
u, v = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
x = np.cos(u) * np.sin(v)
y = np.sin(u) * np.sin(v)
z = np.cos(v)
ax.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, alpha=0.25, antialiased=False)
ax.plot(pnts[0], pnts[1], pnts[2], color='r', lw=4.0, label='Exact')
ax.set_title('Net Magnetization', fontsize=20)
plt.axis('off')
plt.show()

In [ ]:
pnts[2]